In [ ]:
# Grab data from natural earth
zip_url = "http://www.naturalearthdata.com/http//www.naturalearthdata.com/download/10m/cultural/ne_10m_admin_0_map_subunits.zip"
# http://ec.europa.eu/eurostat/statistics-explained/index.php/Glossary:Country_codes

In [ ]:
# Download the natural earth dataset, and exrtact files
from StringIO import StringIO
from zipfile import ZipFile
from urllib import urlopen

url = urlopen(zip_url)
zipfile = ZipFile(StringIO(url.read()))

for name in zipfile.namelist():
    outpath = r"natural_earth_data/"
    zipfile.extract(name, outpath)

In [ ]:
%%bash
shp2pgsql -I -s 4326 natural_earth_data/ne_10m_admin_0_map_subunits.shp public.natural_earth_admin_subunits| psql -d postgres

In [ ]:
# Run a query that aggregates all units to soverign except uk
import psycopg2
con_string = "host='localhost' dbname='postgres' user='postgres' password=''"
conn = psycopg2.connect(con_string)
cursor = conn.cursor()

import pandas as pd
pd.options.display.max_columns = 10000
# view table
sql = "select * from natural_earth_admin_subunits where  sovereignt = 'Denmark' limit 5"
pd.read_sql(sql,conn)

In [ ]:
sql = """
drop table if exists brexit_country_map;
create table brexit_country_map as
select sovereignt,geounit as name, gu_a3 as code, continent, subregion, sum(pop_est) as sum_est_pop, ST_Union(ST_snaptogrid(geom,0.0001))  as geom from natural_earth_admin_subunits 
where sovereignt = 'United Kingdom'
group by geounit, gu_a3, continent, subregion,sovereignt
union all
select sovereignt,admin as name, adm0_a3 as code, continent, subregion, sum(pop_est) as sum_est_pop,  ST_Union(ST_snaptogrid(geom,0.0001))  as geom from natural_earth_admin_subunits 
where sovereignt != 'United Kingdom'
group by admin, adm0_a3, continent, subregion,sovereignt
"""
conn.rollback()
cursor.execute(sql)
conn.commit()

In [ ]:
sql = """
select sovereignt,name,code,continent,subregion,sum_est_pop from brexit_country_map limit 5
"""
pd.read_sql(sql,conn)

In [ ]:
# Now want to add a column that includes the EU countries
# from http://ec.europa.eu/eurostat/statistics-explained/index.php/Glossary:Country_codes
list_of_countries = """
Belgium	BE
Greece	EL
Lithuania	LT
Portugal	PT
Bulgaria	BG
Spain	ES
Luxembourg	LU
Romania	RO
Czech Republic	CZ
France	FR
Hungary	HU
Slovenia	SI
Denmark	DK
Croatia	HR
Malta	MT
Slovakia	SK
Germany	DE
Italy	IT
Netherlands	NL
Finland	FI
Estonia	EE
Cyprus	CY
Austria	AT
Sweden	SE
Ireland	IE
Latvia	LV
Poland	PL
United Kingdom	UK
"""
EU_countries = pd.DataFrame([a.split("\t") for a in list_of_countries.split("\n") if a != ""])
EU_countries.columns =["eu_country_name", "eu_country_code"]

In [ ]:
# 

sql = """
ALTER TABLE brexit_country_map ADD COLUMN eu_country boolean ;
"""
cursor.execute(sql)
conn.commit()

In [ ]:
c =list(EU_countries["eu_country_name"])
c = ["'" + a + "'" for a in c]
isin_list = ', '.join(c)

In [ ]:
sql = """
UPDATE brexit_country_map  SET eu_country = True  WHERE sovereignt in ({}) and continent= 'Europe';
"""
cursor.execute(sql.format(isin_list))
conn.commit()

In [ ]:
sql = """
UPDATE brexit_country_map  SET eu_country = False  WHERE not (sovereignt in ({}) and continent= 'Europe');
"""
cursor.execute(sql.format(isin_list))
conn.commit()

In [ ]:
# Now we convert the postgis to geojson
import psycopg2
con_string = "host='localhost' dbname='postgres' user='postgres' password=''"
conn = psycopg2.connect(con_string)
cursor = conn.cursor()


In [ ]:
# Delete file if exists
import os


In [ ]:
%%bash
rm natural_earth_data/subunits.geojson
ogr2ogr -f "GeoJSON" natural_earth_data/subunits.geojson  PG:"host='localhost' port=5432 user='postgres' dbname='postgres' user='postgres' password=''"  "public.brexit_country_map"

In [ ]:
",".join([i + "=" + i.upper() for i in "sovereignt	name	code	continent	subregion	sum_est_pop".split("\t")])

In [ ]:
%%bash
topojson \
  -o /Users/robinlinacre/Documents/visualisation_projects/brexit_maps/topojson/brexit_countries.json \
  --id-property CODE \
  --simplify-proportion 0.4\
  --stitch-poles false \
  --properties name=NAME,code=CODE,continent=CONTINENT,subregion=SUBREGION,sum_est_pop=SUM_EST_POP  \
  -- natural_earth_data/subunits.geojson 

In [ ]:
# Now we need to do the same with the HMCTS regions data
zip_url = "https://github.com/RobinL/hmcts_regions_shapefile_and_lookup/raw/master/district_borough_hmcts_regions.zip"

In [ ]:
# Download the natural earth dataset, and exrtact files

url = urlopen(zip_url)
zipfile = ZipFile(StringIO(url.read()))

for name in zipfile.namelist():
    outpath = r"hmcts_regions/"
    zipfile.extract(name, outpath)

In [ ]:
%%bash
shp2pgsql -I -s 27700 hmcts_regions/district_borough_hmcts_regions.shp public.hmcts_regions| psql -d postgres

In [ ]:
# Want only hmcts regions, scotland and NI
sql = """
drop table if exists hmcts_regions_merged;

create table hmcts_regions_merged as

select hmcts as hmcts_region, st_transform(st_union(geom),4326)  from hmcts_regions
where hmcts is not null
group by hmcts
union all


select name, geom
from brexit_country_map
where name in ('Northern Ireland', 'Scotland')

"""
conn.rollback()
cursor.execute(sql)
conn.commit()

In [ ]:
%%bash
rm hmcts_regions/hmcts_regions_merged.geojson
ogr2ogr -f "GeoJSON" hmcts_regions/subunits.geojson  PG:"host='localhost' port=5432 user='postgres' dbname='postgres' user='postgres' password=''"  "public.hmcts_regions_merged"

In [ ]:
%%bash
topojson \
  -o /Users/robinlinacre/Documents/visualisation_projects/brexit_maps/topojson/hmcts_regions.json \
  --id-property CODE \
  --simplify-proportion 0.4\
  --stitch-poles false \
  --properties name=NAME,code=CODE,continent=CONTINENT,subregion=SUBREGION,sum_est_pop=SUM_EST_POP  \
  -- hmcts_regions/subunits.geojson 